In [1]:
import torch
import torchvision
import torch.optim as optim
from torch.autograd import Variable
from image_helper import *
from parse_xml_annotations import *
from features import *
from reinforcement import *
from metrics import *
from collections import namedtuple
import time
import os
import numpy as np
import random



path_voc = "C:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2007"

# get models 
print("load models")

model_vgg = getVGG_16bn("../models")
model_vgg = model_vgg.cuda()
model = get_q_network()
model = model.cuda()

# define optimizers for each model
optimizer = optim.Adam(model.parameters(),lr=1e-6)
criterion = nn.MSELoss().cuda()   


load models


d:\coding\Pytorch_Deep_RL_1-master\Pytorch_Deep_RL_1-master\reinforcement.py:84: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data)


In [2]:
# get image datas
path_voc_1 = "C:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2007"
path_voc_2 = "C:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2012"
class_object = 'dog' #aeroplane(1) dog(12)
image_names_1, images_1 = load_image_data(path_voc_1, class_object)
image_names_2, images_2 = load_image_data(path_voc_2, class_object)
image_names = image_names_1 + image_names_2
images = images_1 + images_2

print("aeroplane_trainval image:%d" % len(image_names))

# define the Pytorch Tensor
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

load imagesC:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2007
total image:421
load imagesC:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2012
total image:1286
aeroplane_trainval image:1707


In [3]:
# define the super parameter
epsilon = 1.0
BATCH_SIZE = 100
GAMMA = 0.90
CLASS_OBJECT = 1
steps = 14
change = 4
epochs = 50
alpha = 0.2
memory = ReplayMemory(1000)

In [4]:
def select_h_action(state):
    if random.random() < epsilon:
        action = np.random.randint(1,7)
    else:
        qval = model(Variable(state))
        _, predicted = torch.max(qval.data,1)
        action = predicted[0] + 1
    return action

In [5]:
def select_d_action(self, state, actions, ground_truth):
        """
            Selection de l'action dépendemment de l'état
            상태에 따른 동작 선택
            Entrée :
                - Etat actuel. 
                - Vérité terrain.
            Sortie :
                - Soi l'action qu'aura choisi le modèle soi la meilleure action possible ( Le choix entre les deux se fait selon un jet aléatoire ).
        """
        sample = random.random()
        eps_threshold = self.EPS
        self.steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                if use_cuda:
                    inpu = Variable(state).cuda()
                else:
                    inpu = Variable(state)
                qval = self.policy_net(inpu)
                _, predicted = torch.max(qval.data,1)
                action = predicted[0] # + 1
                try:
                  return action.cpu().numpy()[0]
                except:
                  return action.cpu().numpy()
        else:
            #return np.random.randint(0,9)   # Avant implémentation d'agent expert
            return get_best_next_action(actions, ground_truth) # Appel à l'agent expert.

In [6]:
def get_best_next_action(self, actions, ground_truth):
        """
            Implémentation de l'Agent expert qui selon l'état actuel et la vérité terrain va donner la meilleur action possible.
            현재 상태와 실제 상황에 따라 가능한 최선의 조치를 제공하는 Expert Agent 구현.
            Entrée :
                - Liste d'actions executées jusqu'à présent.
                - Vérité terrain.
            Sortie :
                - Indice de la meilleure action possible.

        """
        max_reward = -99
        best_action = -99
        positive_actions = []
        negative_actions = []
        actual_equivalent_coord = self.calculate_position_box(actions)
        for i in range(0, 9):
            copy_actions = actions.copy()
            copy_actions.append(i)
            new_equivalent_coord = self.calculate_position_box(copy_actions)
            if i!=0:
                reward = self.compute_reward(new_equivalent_coord, actual_equivalent_coord, ground_truth)
            else:
                reward = self.compute_trigger_reward(new_equivalent_coord,  ground_truth)
            
            if reward>=0:
                positive_actions.append(i)
            else:
                negative_actions.append(i)
        if len(positive_actions)==0:
            return random.choice(negative_actions)
        return random.choice(positive_actions)

In [7]:
def calculate_position_box(self, actions, xmin=0, xmax=224, ymin=0, ymax=224):
        """
            Prends l'ensemble des actions depuis le début et en génére les coordonnées finales de la boite englobante.
            처음부터 모든 작업을 수행하고 경계 상자의 최종 좌표를 생성합니다.
            Entrée :
                - Ensemble des actions sélectionnées depuis le début.
            Sortie :
                - Coordonnées finales de la boite englobante.
        """
        # Calcul des alpha_h et alpha_w mentionnées dans le papier
        alpha_h = self.alpha * (  ymax - ymin )
        alpha_w = self.alpha * (  xmax - xmin )
        real_x_min, real_x_max, real_y_min, real_y_max = 0, 224, 0, 224

        # Boucle sur l'ensemble des actions
        for r in actions:
            if r == 1: # Right
                real_x_min += alpha_w
                real_x_max += alpha_w
            if r == 2: # Left
                real_x_min -= alpha_w
                real_x_max -= alpha_w
            if r == 3: # Up 
                real_y_min -= alpha_h
                real_y_max -= alpha_h
            if r == 4: # Down
                real_y_min += alpha_h
                real_y_max += alpha_h
            if r == 5: # Bigger
                real_y_min -= alpha_h
                real_y_max += alpha_h
                real_x_min -= alpha_w
                real_x_max += alpha_w
            if r == 6: # Smaller
                real_y_min += alpha_h
                real_y_max -= alpha_h
                real_x_min += alpha_w
                real_x_max -= alpha_w
            if r == 7: # Fatter
                real_y_min += alpha_h
                real_y_max -= alpha_h
            if r == 8: # Taller
                real_x_min += alpha_w
                real_x_max -= alpha_w
        real_x_min, real_x_max, real_y_min, real_y_max = self.rewrap(real_x_min), self.rewrap(real_x_max), self.rewrap(real_y_min), self.rewrap(real_y_max)
        return [real_x_min, real_x_max, real_y_min, real_y_max]

In [8]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))
def optimizer_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
    next_states = [s for s in batch.next_state if s is not None]
    non_final_next_states = Variable(torch.cat(next_states), 
                                     volatile=True).type(Tensor)
    state_batch = Variable(torch.cat(batch.state)).type(Tensor)
    action_batch = Variable(torch.LongTensor(batch.action).view(-1,1)).type(LongTensor)
    reward_batch = Variable(torch.FloatTensor(batch.reward).view(-1,1)).type(Tensor)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    state_action_values = model(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    next_state_values = Variable(torch.zeros(BATCH_SIZE, 1).type(Tensor)) 
    next_state_values[non_final_mask.view(-1,1)] = model(non_final_next_states).max(1)[0]
    
    # Now, we don't want to mess up the loss with a volatile flag, so let's
    # clear it. After this, we'll just end up with a Variable that has
    # requires_grad=False
    next_state_values.volatile = False
    
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute  loss
    loss = criterion(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [9]:
def compose_state(self, image, dtype=FloatTensor):
        """
            Composition d'un état : Feature Vector + Historique des actions
            상태 구성: 기능 벡터 + 작업 내역
            Entrée :
                - Image ( feature vector ). 
            Sortie :
                - Représentation d'état.
        """
        image_feature = get_features(image, dtype)
        image_feature = image_feature.view(1,-1)
        #print("image feature : "+str(image_feature.shape))
        history_flatten = self.actions_history.view(1,-1).type(dtype)
        state = torch.cat((image_feature, history_flatten), 1)
        return state

In [10]:
def get_features(self, image, dtype=FloatTensor):
        """
            Extraction du feature vector à partir de l'image.
            이미지에서 특징 벡터를 추출합니다.
            Entrée :
                - Image
            Sortie :
                - Feature vector
        """
    
        global transform
        #image = transform(image)
        image = image.view(1,*image.shape)
        image = Variable(image).type(dtype)
        if use_cuda:
            image = image.cuda()
        feature = self.feature_extractor(image)
        #print("Feature shape : "+str(feature.shape))
        return feature.data